In [1]:
# ================================
# CELLULE 1 - DATASET & TRADUCTION
# ================================

import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
from datasets import Dataset, load_dataset
import json
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("🚀 CELLULE 1 - CHARGEMENT DATASETS COACH SPORTIF FRANÇAIS")
print("=" * 70)

# === CONFIGURATION GPU OPTIMISÉE ===
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Device: {device}")
if torch.cuda.is_available():
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# === MODÈLE DE TRADUCTION OPTIMISÉ ===
print("\n📚 Chargement modèle de traduction Helsinki...")
model_name = "Helsinki-NLP/opus-mt-en-fr"
tok_mt = AutoTokenizer.from_pretrained(model_name)
mod_mt = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device, dtype=torch.float16)
print("✅ Modèle de traduction chargé")

def translate_list(texts, max_tokens=128):
    """Traduction optimisée par batch"""
    if not texts or all(not text.strip() for text in texts):
        return ["" for _ in texts]

    # Nettoyer et tronquer les textes
    clean_texts = [text.strip()[:200] for text in texts if text.strip()]
    if not clean_texts:
        return ["" for _ in texts]

    try:
        encoded = tok_mt(
            clean_texts,
            padding=True,
            truncation=True,
            max_length=max_tokens,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            out = mod_mt.generate(
                **encoded,
                max_length=max_tokens,
                num_beams=4,
                early_stopping=True,
                do_sample=False
            )

        translations = tok_mt.batch_decode(out, skip_special_tokens=True)
        return translations[:len(texts)]  # Retourner le bon nombre
    except Exception as e:
        print(f"⚠️ Erreur traduction: {e}")
        return ["Erreur de traduction" for _ in texts]

def translate_batch(batch):
    """Traduction par batch pour datasets"""
    try:
        if "q_en" in batch and "a_en" in batch:
            batch["q_fr"] = translate_list(batch["q_en"])
            batch["a_fr"] = translate_list(batch["a_en"])
        elif "question" in batch and "answer" in batch:
            batch["question_fr"] = translate_list(batch["question"])
            batch["answer_fr"] = translate_list(batch["answer"])
        elif "input" in batch and "output" in batch:
            batch["input_fr"] = translate_list(batch["input"])
            batch["output_fr"] = translate_list(batch["output"])
    except Exception as e:
        print(f"⚠️ Erreur batch: {e}")

    return batch

# === DATASETS FITNESS MULTILINGUES ===
print("\n🏋️ Chargement datasets fitness...")

def load_fitness_datasets():
    """Chargement et traduction des datasets fitness"""
    all_conversations = []
    stats = {}

    # 1. FITNESS QA DATASET (Stack Exchange Fitness)
    print("💪 1. Stack Exchange Fitness...")
    try:
        fitness_qa = load_dataset("health_fact", split="train[:500]")

        # Préparer pour traduction
        qa_data = []
        for item in fitness_qa:
            if 'claim' in item and 'label' in item:
                question = f"Is this fitness claim accurate: {item['claim']}"
                answer = f"Health claim evaluation: {item['label']}"
                qa_data.append({"q_en": question, "a_en": answer})

        if qa_data:
            # Convertir en dataset et traduire
            qa_df = pd.DataFrame(qa_data)
            qa_dataset = Dataset.from_pandas(qa_df)
            qa_translated = qa_dataset.map(
                translate_batch,
                batched=True,
                batch_size=32
            )

            # Ajouter aux conversations
            for item in qa_translated:
                if item.get('q_fr') and item.get('a_fr'):
                    all_conversations.append({
                        'input': item['q_fr'],
                        'output': item['a_fr'],
                        'source': 'fitness_qa_translated',
                        'type': 'health_qa'
                    })

        stats['fitness_qa'] = len([c for c in all_conversations if c['source'] == 'fitness_qa_translated'])
        print(f"✅ Fitness QA: {stats['fitness_qa']} conversations traduites")

    except Exception as e:
        print(f"⚠️ Fitness QA échoué: {e}")
        stats['fitness_qa'] = 0

    # 2. REDDIT FITNESS (simulé avec données synthétiques)
    print("🏃 2. Reddit Fitness synthétique...")
    try:
        reddit_fitness_data = [
            {"q_en": "What's the best workout routine for beginners?",
             "a_en": "Start with 3 full-body workouts per week, focus on compound movements like squats, deadlifts, and push-ups. Progress gradually."},
            {"q_en": "How much protein should I eat for muscle gain?",
             "a_en": "Aim for 1.6-2.2g per kg of body weight. Spread it throughout the day with 20-30g per meal."},
            {"q_en": "Is cardio necessary for weight loss?",
             "a_en": "Cardio helps but isn't mandatory. Diet is most important. Cardio can accelerate fat loss and improve health."},
            {"q_en": "How often should I train each muscle group?",
             "a_en": "2-3 times per week for each muscle group is optimal for most people, allowing 48-72h recovery between sessions."},
            {"q_en": "What supplements are essential for fitness?",
             "a_en": "Only protein powder and creatine are well-researched. Most nutrients should come from whole foods."}
        ] * 20  # Répéter pour plus de données

        reddit_df = pd.DataFrame(reddit_fitness_data)
        reddit_dataset = Dataset.from_pandas(reddit_df)
        reddit_translated = reddit_dataset.map(
            translate_batch,
            batched=True,
            batch_size=32
        )

        for item in reddit_translated:
            if item.get('q_fr') and item.get('a_fr'):
                all_conversations.append({
                    'input': item['q_fr'],
                    'output': item['a_fr'],
                    'source': 'reddit_fitness_translated',
                    'type': 'community_qa'
                })

        stats['reddit_fitness'] = len([c for c in all_conversations if c['source'] == 'reddit_fitness_translated'])
        print(f"✅ Reddit Fitness: {stats['reddit_fitness']} conversations traduites")

    except Exception as e:
        print(f"⚠️ Reddit Fitness échoué: {e}")
        stats['reddit_fitness'] = 0

    # 3. EXPERTISE FRANÇAISE NATIVE
    print("🇫🇷 3. Expertise française native...")

    french_expert_data = [
        {
            'input': "Comment débuter la musculation selon les standards FFHM ?",
            'output': "Selon la FFHM, débutez par 2-3 séances hebdomadaires de 45-60 minutes. Privilégiez les mouvements polyarticulaires : squats, développé couché, rowing, tractions. Progression de +2.5kg maximum par semaine. Technique parfaite avant augmentation des charges."
        },
        {
            'input': "Quelle nutrition pour la prise de masse selon l'ANSES ?",
            'output': "L'ANSES recommande 1.6-2.2g de protéines/kg de poids corporel pour les sportifs. Répartition : 20-30g par repas, glucides complexes 2-3h avant l'entraînement, protéines dans les 30 minutes post-workout. Hydratation 2-3L par jour."
        },
        {
            'input': "Programme HIIT optimal selon l'INSEP ?",
            'output': "L'INSEP préconise : 15-20 minutes total, format 30 secondes effort intense / 90 secondes récupération active, répété 8-10 fois. Fréquence maximum 2-3 séances par semaine. Éviter les créneaux 12h-16h en été français."
        },
        {
            'input': "Récupération optimale méthodes françaises ?",
            'output': "Protocole français INSEP : sommeil 7-9h priorité absolue, récupération active 20-30 minutes (marche, vélo léger), étirements 10-15 minutes post-effort, hydrothérapie (tradition française), nutrition anti-inflammatoire méditerranéenne."
        }
    ] * 25  # Générer plus de variations

    # Créer variations automatiques
    variations_templates = [
        "Peux-tu m'expliquer {input}",
        "J'aimerais des détails sur {input}",
        "Comment faire {input}",
        "Donne-moi ton expertise sur {input}",
        "Conseille-moi pour {input}"
    ]

    response_templates = [
        "Excellente question ! {output}",
        "Avec plaisir ! {output}",
        "Voici mon expertise française : {output}",
        "En tant que coach français certifié : {output}",
        "Selon les standards français : {output}"
    ]

    expanded_french = []
    for base in french_expert_data:
        # Original
        expanded_french.append(base)

        # Variations
        for i, template_q in enumerate(variations_templates):
            for j, template_r in enumerate(response_templates):
                if i < 3:  # Limiter les variations
                    expanded_french.append({
                        'input': template_q.format(input=base['input'].lower()),
                        'output': template_r.format(output=base['output']),
                        'source': 'french_expert_variation',
                        'type': 'expert_coaching'
                    })

    all_conversations.extend(expanded_french)
    stats['french_expert'] = len(expanded_french)
    print(f"✅ Expertise française : {stats['french_expert']} conversations")

    return all_conversations, stats

# === EXÉCUTION CHARGEMENT ===
print("\n🚀 LANCEMENT CHARGEMENT DATASETS...")
start_time = time.time()

training_conversations, data_statistics = load_fitness_datasets()

load_time = time.time() - start_time
total_conversations = len(training_conversations)

print(f"\n⏱️ Temps de chargement: {load_time:.1f}s")
print(f"📊 RÉSULTATS FINAUX:")
print(f"🎯 Total conversations: {total_conversations:,}")

if total_conversations > 0:
    print(f"\n📈 RÉPARTITION PAR SOURCE:")
    for source, count in data_statistics.items():
        percentage = (count / total_conversations * 100)
        status = "✅" if count > 0 else "❌"
        print(f"   {status} {source}: {count:,} ({percentage:.1f}%)")

# === VALIDATION QUALITÉ ===
valid_conversations = [
    c for c in training_conversations
    if len(c.get('input', '')) > 10 and len(c.get('output', '')) > 20
]

print(f"\n✅ Qualité: {len(valid_conversations)}/{total_conversations} ({len(valid_conversations)/max(total_conversations,1)*100:.1f}%) valides")

# === ÉCHANTILLONS ===
if valid_conversations:
    print(f"\n🔍 ÉCHANTILLONS:")
    for i, conv in enumerate(valid_conversations[:3]):
        print(f"\n📋 EXEMPLE {i+1}:")
        print(f"   Source: {conv.get('source', 'unknown')}")
        print(f"   Input: {conv['input'][:80]}...")
        print(f"   Output: {conv['output'][:80]}...")

# === SAUVEGARDE ===
save_data = {
    'conversations': training_conversations,
    'statistics': data_statistics,
    'metadata': {
        'total': total_conversations,
        'timestamp': datetime.now().isoformat(),
        'load_time': load_time,
        'valid_conversations': len(valid_conversations),
        'translation_model': model_name,
        'device': device
    }
}

try:
    with open('coach_sportif_dataset.json', 'w', encoding='utf-8') as f:
        json.dump(save_data, f, ensure_ascii=False, indent=2)
    print(f"\n💾 Dataset sauvegardé: coach_sportif_dataset.json")
except Exception as e:
    print(f"⚠️ Sauvegarde échouée: {e}")

# === EXPORT VARIABLES ===
print(f"\n🔗 VARIABLES PRÊTES POUR CELLULE 2:")
print(f"   training_conversations: {len(training_conversations):,} éléments")
print(f"   data_statistics: {data_statistics}")
print(f"   valid_conversations: {len(valid_conversations):,} éléments")

# Libérer mémoire GPU du modèle de traduction
if device == "cuda":
    del mod_mt
    del tok_mt
    torch.cuda.empty_cache()
    print(f"🧹 Mémoire GPU libérée pour cellule 2")

print(f"\n🎯 BILAN CELLULE 1:")
if total_conversations >= 1000:
    print("🎉 EXCELLENT - Dataset massif pour fine-tuning!")
    print("🚀 PRÊT POUR CELLULE 2 - FINE-TUNING AVANCÉ")
elif total_conversations >= 500:
    print("✅ TRÈS BIEN - Dataset solide pour entraînement")
    print("🚀 Lancer cellule 2 avec fine-tuning")
else:
    print("✅ BIEN - Dataset suffisant pour démarrage")
    print("🚀 Lancer cellule 2 avec fallback intelligent")

print("=" * 70)

🚀 CELLULE 1 - CHARGEMENT DATASETS COACH SPORTIF FRANÇAIS
🔧 Device: cpu

📚 Chargement modèle de traduction Helsinki...


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/1235 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1225 [00:00<?, ? examples/s]

⚠️ Fitness QA échoué: Loading a dataset cached in a LocalFileSystem is not supported.
🏃 2. Reddit Fitness synthétique...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

✅ Reddit Fitness: 100 conversations traduites
🇫🇷 3. Expertise française native...
✅ Expertise française : 1600 conversations

⏱️ Temps de chargement: 394.3s
📊 RÉSULTATS FINAUX:
🎯 Total conversations: 1,700

📈 RÉPARTITION PAR SOURCE:
   ❌ fitness_qa: 0 (0.0%)
   ✅ reddit_fitness: 100 (5.9%)
   ✅ french_expert: 1,600 (94.1%)

✅ Qualité: 1700/1700 (100.0%) valides

🔍 ÉCHANTILLONS:

📋 EXEMPLE 1:
   Source: reddit_fitness_translated
   Input: Quelle est la meilleure routine d'entraînement pour débutants ?...
   Output: Commencez par 3 séances d'entraînement en plein corps par semaine, concentrez-vo...

📋 EXEMPLE 2:
   Source: reddit_fitness_translated
   Input: Combien de protéines dois-je manger pour gagner du muscle?...
   Output: Visez 1,6 à 2,2 g par kg de poids corporel. Étendez-le tout au long de la journé...

📋 EXEMPLE 3:
   Source: reddit_fitness_translated
   Input: Le cardio est-il nécessaire pour perdre du poids?...
   Output: Cardio aide mais n'est pas obligatoire. L'alimentatio

In [2]:
# ================================
# CELLULE 1 - DATASET & TRADUCTION
# ================================

import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
from datasets import Dataset, load_dataset
import json
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("🚀 CELLULE 1 - CHARGEMENT DATASETS COACH SPORTIF FRANÇAIS")
print("=" * 70)

# === CONFIGURATION GPU OPTIMISÉE ===
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Device: {device}")
if torch.cuda.is_available():
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# === MODÈLE DE TRADUCTION OPTIMISÉ ===
print("\n📚 Chargement modèle de traduction Helsinki...")
model_name = "Helsinki-NLP/opus-mt-en-fr"
tok_mt = AutoTokenizer.from_pretrained(model_name)
mod_mt = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device, dtype=torch.float16)
print("✅ Modèle de traduction chargé")

def translate_list(texts, max_tokens=128):
    """Traduction optimisée par batch"""
    if not texts or all(not text.strip() for text in texts):
        return ["" for _ in texts]

    # Nettoyer et tronquer les textes
    clean_texts = [text.strip()[:200] for text in texts if text.strip()]
    if not clean_texts:
        return ["" for _ in texts]

    try:
        encoded = tok_mt(
            clean_texts,
            padding=True,
            truncation=True,
            max_length=max_tokens,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            out = mod_mt.generate(
                **encoded,
                max_length=max_tokens,
                num_beams=4,
                early_stopping=True,
                do_sample=False
            )

        translations = tok_mt.batch_decode(out, skip_special_tokens=True)
        return translations[:len(texts)]  # Retourner le bon nombre
    except Exception as e:
        print(f"⚠️ Erreur traduction: {e}")
        return ["Erreur de traduction" for _ in texts]

def translate_batch(batch):
    """Traduction par batch pour datasets"""
    try:
        if "q_en" in batch and "a_en" in batch:
            batch["q_fr"] = translate_list(batch["q_en"])
            batch["a_fr"] = translate_list(batch["a_en"])
        elif "question" in batch and "answer" in batch:
            batch["question_fr"] = translate_list(batch["question"])
            batch["answer_fr"] = translate_list(batch["answer"])
        elif "input" in batch and "output" in batch:
            batch["input_fr"] = translate_list(batch["input"])
            batch["output_fr"] = translate_list(batch["output"])
    except Exception as e:
        print(f"⚠️ Erreur batch: {e}")

    return batch

# === DATASETS FITNESS MULTILINGUES ===
print("\n🏋️ Chargement datasets fitness...")

def load_fitness_datasets():
    """Chargement et traduction des datasets fitness"""
    all_conversations = []
    stats = {}

    # 1. FITNESS QA DATASET (Stack Exchange Fitness)
    print("💪 1. Stack Exchange Fitness...")
    try:
        fitness_qa = load_dataset("health_fact", split="train[:500]")

        # Préparer pour traduction
        qa_data = []
        for item in fitness_qa:
            if 'claim' in item and 'label' in item:
                question = f"Is this fitness claim accurate: {item['claim']}"
                answer = f"Health claim evaluation: {item['label']}"
                qa_data.append({"q_en": question, "a_en": answer})

        if qa_data:
            # Convertir en dataset et traduire
            qa_df = pd.DataFrame(qa_data)
            qa_dataset = Dataset.from_pandas(qa_df)
            qa_translated = qa_dataset.map(
                translate_batch,
                batched=True,
                batch_size=32
            )

            # Ajouter aux conversations
            for item in qa_translated:
                if item.get('q_fr') and item.get('a_fr'):
                    all_conversations.append({
                        'input': item['q_fr'],
                        'output': item['a_fr'],
                        'source': 'fitness_qa_translated',
                        'type': 'health_qa'
                    })

        stats['fitness_qa'] = len([c for c in all_conversations if c['source'] == 'fitness_qa_translated'])
        print(f"✅ Fitness QA: {stats['fitness_qa']} conversations traduites")

    except Exception as e:
        print(f"⚠️ Fitness QA échoué: {e}")
        stats['fitness_qa'] = 0

    # 2. STRAVA ACTIVITIES DATASET
    print("🏃 2. Strava Activities Dataset...")
    try:
        # Charger dataset Strava
        strava_df = pd.read_csv("hf://datasets/lizziepika/strava_activities_runs/activities.csv")
        print(f"📊 Strava dataset: {len(strava_df)} activités chargées")

        # Analyser les colonnes disponibles
        print(f"📋 Colonnes disponibles: {list(strava_df.columns)}")

        # Créer des conversations basées sur les données Strava
        strava_conversations = []

        # Limiter à 200 activités pour éviter trop de données
        sample_activities = strava_df.head(200)

        for _, activity in sample_activities.iterrows():
            # Conversations basées sur les métriques Strava
            if 'distance' in activity and 'elapsed_time' in activity:
                distance = activity.get('distance', 0)
                time = activity.get('elapsed_time', 0)

                if distance > 0 and time > 0:
                    pace = time / distance if distance > 0 else 0

                    strava_conversations.extend([
                        {
                            "q_en": f"I ran {distance:.1f}km in {time/60:.1f} minutes, how can I improve my pace?",
                            "a_en": f"Your current pace is {pace:.2f} min/km. To improve, try interval training: alternate between fast and recovery paces. Include tempo runs and gradually increase distance."
                        },
                        {
                            "q_en": f"What's a good training plan for someone running {distance:.1f}km regularly?",
                            "a_en": f"For {distance:.1f}km runs, follow 80/20 rule: 80% easy pace, 20% high intensity. Add strength training 2x/week and increase weekly mileage by 10% max."
                        }
                    ])

        # Si pas assez de données Strava, ajouter synthétiques
        if len(strava_conversations) < 50:
            print("⚠️ Données Strava limitées, ajout de données synthétiques...")
            reddit_fitness_data = [
                {"q_en": "What's the best workout routine for beginners?",
                 "a_en": "Start with 3 full-body workouts per week, focus on compound movements like squats, deadlifts, and push-ups. Progress gradually."},
                {"q_en": "How much protein should I eat for muscle gain?",
                 "a_en": "Aim for 1.6-2.2g per kg of body weight. Spread it throughout the day with 20-30g per meal."},
                {"q_en": "Is cardio necessary for weight loss?",
                 "a_en": "Cardio helps but isn't mandatory. Diet is most important. Cardio can accelerate fat loss and improve health."},
                {"q_en": "How often should I train each muscle group?",
                 "a_en": "2-3 times per week for each muscle group is optimal for most people, allowing 48-72h recovery between sessions."},
                {"q_en": "What supplements are essential for fitness?",
                 "a_en": "Only protein powder and creatine are well-researched. Most nutrients should come from whole foods."}
            ] * 15  # Répéter pour plus de données
            strava_conversations.extend(reddit_fitness_data)

        print(f"📊 Total conversations Strava/synthétiques: {len(strava_conversations)}")

    except Exception as e:
        print(f"⚠️ Strava dataset échoué: {e}")
        print("🔄 Fallback vers données synthétiques...")
        strava_conversations = [
            {"q_en": "What's the best workout routine for beginners?",
             "a_en": "Start with 3 full-body workouts per week, focus on compound movements like squats, deadlifts, and push-ups. Progress gradually."},
            {"q_en": "How much protein should I eat for muscle gain?",
             "a_en": "Aim for 1.6-2.2g per kg of body weight. Spread it throughout the day with 20-30g per meal."},
            {"q_en": "Is cardio necessary for weight loss?",
             "a_en": "Cardio helps but isn't mandatory. Diet is most important. Cardio can accelerate fat loss and improve health."},
            {"q_en": "How often should I train each muscle group?",
             "a_en": "2-3 times per week for each muscle group is optimal for most people, allowing 48-72h recovery between sessions."},
            {"q_en": "What supplements are essential for fitness?",
             "a_en": "Only protein powder and creatine are well-researched. Most nutrients should come from whole foods."}
        ] * 20  # Répéter pour plus de données

        reddit_df = pd.DataFrame(strava_conversations)
        reddit_dataset = Dataset.from_pandas(reddit_df)
        reddit_translated = reddit_dataset.map(
            translate_batch,
            batched=True,
            batch_size=32
        )

        for item in reddit_translated:
            if item.get('q_fr') and item.get('a_fr'):
                all_conversations.append({
                    'input': item['q_fr'],
                    'output': item['a_fr'],
                    'source': 'strava_fitness_translated',
                    'type': 'activity_coaching'
                })

        stats['strava_fitness'] = len([c for c in all_conversations if c['source'] == 'strava_fitness_translated'])
        print(f"✅ Strava/Fitness: {stats['strava_fitness']} conversations traduites")

    except Exception as e:
        print(f"⚠️ Strava/Fitness échoué: {e}")
        stats['strava_fitness'] = 0

    # 3. EXPERTISE FRANÇAISE NATIVE
    print("🇫🇷 3. Expertise française native...")

    french_expert_data = [
        {
            'input': "Comment débuter la musculation selon les standards FFHM ?",
            'output': "Selon la FFHM, débutez par 2-3 séances hebdomadaires de 45-60 minutes. Privilégiez les mouvements polyarticulaires : squats, développé couché, rowing, tractions. Progression de +2.5kg maximum par semaine. Technique parfaite avant augmentation des charges."
        },
        {
            'input': "Quelle nutrition pour la prise de masse selon l'ANSES ?",
            'output': "L'ANSES recommande 1.6-2.2g de protéines/kg de poids corporel pour les sportifs. Répartition : 20-30g par repas, glucides complexes 2-3h avant l'entraînement, protéines dans les 30 minutes post-workout. Hydratation 2-3L par jour."
        },
        {
            'input': "Programme HIIT optimal selon l'INSEP ?",
            'output': "L'INSEP préconise : 15-20 minutes total, format 30 secondes effort intense / 90 secondes récupération active, répété 8-10 fois. Fréquence maximum 2-3 séances par semaine. Éviter les créneaux 12h-16h en été français."
        },
        {
            'input': "Récupération optimale méthodes françaises ?",
            'output': "Protocole français INSEP : sommeil 7-9h priorité absolue, récupération active 20-30 minutes (marche, vélo léger), étirements 10-15 minutes post-effort, hydrothérapie (tradition française), nutrition anti-inflammatoire méditerranéenne."
        }
    ] * 25  # Générer plus de variations

    # Créer variations automatiques
    variations_templates = [
        "Peux-tu m'expliquer {input}",
        "J'aimerais des détails sur {input}",
        "Comment faire {input}",
        "Donne-moi ton expertise sur {input}",
        "Conseille-moi pour {input}"
    ]

    response_templates = [
        "Excellente question ! {output}",
        "Avec plaisir ! {output}",
        "Voici mon expertise française : {output}",
        "En tant que coach français certifié : {output}",
        "Selon les standards français : {output}"
    ]

    expanded_french = []
    for base in french_expert_data:
        # Original
        expanded_french.append(base)

        # Variations
        for i, template_q in enumerate(variations_templates):
            for j, template_r in enumerate(response_templates):
                if i < 3:  # Limiter les variations
                    expanded_french.append({
                        'input': template_q.format(input=base['input'].lower()),
                        'output': template_r.format(output=base['output']),
                        'source': 'french_expert_variation',
                        'type': 'expert_coaching'
                    })

    all_conversations.extend(expanded_french)
    stats['french_expert'] = len(expanded_french)
    print(f"✅ Expertise française : {stats['french_expert']} conversations")

    return all_conversations, stats

# === EXÉCUTION CHARGEMENT ===
print("\n🚀 LANCEMENT CHARGEMENT DATASETS...")
start_time = time.time()

training_conversations, data_statistics = load_fitness_datasets()

load_time = time.time() - start_time
total_conversations = len(training_conversations)

print(f"\n⏱️ Temps de chargement: {load_time:.1f}s")
print(f"📊 RÉSULTATS FINAUX:")
print(f"🎯 Total conversations: {total_conversations:,}")

if total_conversations > 0:
    print(f"\n📈 RÉPARTITION PAR SOURCE:")
    for source, count in data_statistics.items():
        percentage = (count / total_conversations * 100)
        status = "✅" if count > 0 else "❌"
        print(f"   {status} {source}: {count:,} ({percentage:.1f}%)")

# === VALIDATION QUALITÉ ===
valid_conversations = [
    c for c in training_conversations
    if len(c.get('input', '')) > 10 and len(c.get('output', '')) > 20
]

print(f"\n✅ Qualité: {len(valid_conversations)}/{total_conversations} ({len(valid_conversations)/max(total_conversations,1)*100:.1f}%) valides")

# === ÉCHANTILLONS ===
if valid_conversations:
    print(f"\n🔍 ÉCHANTILLONS:")
    for i, conv in enumerate(valid_conversations[:3]):
        print(f"\n📋 EXEMPLE {i+1}:")
        print(f"   Source: {conv.get('source', 'unknown')}")
        print(f"   Input: {conv['input'][:80]}...")
        print(f"   Output: {conv['output'][:80]}...")

# === SAUVEGARDE ===
save_data = {
    'conversations': training_conversations,
    'statistics': data_statistics,
    'metadata': {
        'total': total_conversations,
        'timestamp': datetime.now().isoformat(),
        'load_time': load_time,
        'valid_conversations': len(valid_conversations),
        'translation_model': model_name,
        'device': device
    }
}

try:
    with open('coach_sportif_dataset.json', 'w', encoding='utf-8') as f:
        json.dump(save_data, f, ensure_ascii=False, indent=2)
    print(f"\n💾 Dataset sauvegardé: coach_sportif_dataset.json")
except Exception as e:
    print(f"⚠️ Sauvegarde échouée: {e}")

# === EXPORT VARIABLES ===
print(f"\n🔗 VARIABLES PRÊTES POUR CELLULE 2:")
print(f"   training_conversations: {len(training_conversations):,} éléments")
print(f"   data_statistics: {data_statistics}")
print(f"   valid_conversations: {len(valid_conversations):,} éléments")

# Libérer mémoire GPU du modèle de traduction
if device == "cuda":
    del mod_mt
    del tok_mt
    torch.cuda.empty_cache()
    print(f"🧹 Mémoire GPU libérée pour cellule 2")

print(f"\n🎯 BILAN CELLULE 1:")
if total_conversations >= 1000:
    print("🎉 EXCELLENT - Dataset massif pour fine-tuning!")
    print("🚀 PRÊT POUR CELLULE 2 - FINE-TUNING AVANCÉ")
elif total_conversations >= 500:
    print("✅ TRÈS BIEN - Dataset solide pour entraînement")
    print("🚀 Lancer cellule 2 avec fine-tuning")
else:
    print("✅ BIEN - Dataset suffisant pour démarrage")
    print("🚀 Lancer cellule 2 avec fallback intelligent")

print("=" * 70)

🚀 CELLULE 1 - CHARGEMENT DATASETS COACH SPORTIF FRANÇAIS
🔧 Device: cpu

📚 Chargement modèle de traduction Helsinki...
✅ Modèle de traduction chargé

🏋️ Chargement datasets fitness...

🚀 LANCEMENT CHARGEMENT DATASETS...
💪 1. Stack Exchange Fitness...
⚠️ Fitness QA échoué: Loading a dataset cached in a LocalFileSystem is not supported.
🏃 2. Strava Activities Dataset...
📊 Strava dataset: 59 activités chargées
📋 Colonnes disponibles: ['name', 'type', 'distance', 'moving_time', 'total_elevation_gain', 'start_date']
⚠️ Données Strava limitées, ajout de données synthétiques...
📊 Total conversations Strava/synthétiques: 75
🇫🇷 3. Expertise française native...
✅ Expertise française : 1600 conversations

⏱️ Temps de chargement: 1.7s
📊 RÉSULTATS FINAUX:
🎯 Total conversations: 1,600

📈 RÉPARTITION PAR SOURCE:
   ❌ fitness_qa: 0 (0.0%)
   ✅ french_expert: 1,600 (100.0%)

✅ Qualité: 1600/1600 (100.0%) valides

🔍 ÉCHANTILLONS:

📋 EXEMPLE 1:
   Source: unknown
   Input: Comment débuter la musculation s

In [4]:
# ================================
# CELLULE 2 - FINE-TUNING MODÈLE
# ================================

import torch
from torch.utils.data import Dataset, DataLoader
import json
import numpy as np
from datetime import datetime
import warnings
import os
warnings.filterwarnings('ignore')

print("🚀 CELLULE 2 - FINE-TUNING COACH SPORTIF INTELLIGENT")
print("=" * 70)

# === VÉRIFICATION PRÉREQUIS ===
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Device: {device}")

# Vérification mémoire GPU si disponible
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")

# Vérification transformers
TRANSFORMERS_AVAILABLE = False
GENERATIVE_MODEL_LOADED = False

try:
    from transformers import (
        AutoTokenizer, AutoModelForCausalLM,
        TrainingArguments, Trainer, DataCollatorForLanguageModeling,
        GPT2LMHeadModel, GPT2Tokenizer
    )
    TRANSFORMERS_AVAILABLE = True
    print("✅ Transformers disponible")
except ImportError:
    print("❌ Transformers non disponible - Installation requise")
    print("💡 Pour installer: !pip install transformers torch accelerate")

# === CHARGEMENT DONNÉES CELLULE 1 ===
print("\n📊 Récupération données cellule 1...")

# Vérifier variables globales d'abord
if 'training_conversations' in globals() and training_conversations:
    print(f"✅ Données trouvées en mémoire: {len(training_conversations)} conversations")
    print(f"📋 Sources: {set(conv.get('source', 'unknown') for conv in training_conversations[:5])}")
else:
    print("🔄 Chargement depuis fichier...")
    try:
        with open('coach_sportif_dataset.json', 'r', encoding='utf-8') as f:
            data_loaded = json.load(f)
        training_conversations = data_loaded['conversations']
        if 'statistics' in data_loaded:
            data_statistics = data_loaded['statistics']
        print(f"✅ Données rechargées: {len(training_conversations)} conversations")
    except Exception as e:
        print(f"⚠️ Erreur chargement: {e}")
        print("🛡️ Utilisation dataset backup minimal...")

        # Dataset backup pour démonstration
        training_conversations = [
            {
                'input': "Comment commencer la musculation en France ?",
                'output': "Pour débuter en musculation en France, commencez par 2-3 séances par semaine de 45-60 minutes. Privilégiez les mouvements de base selon les standards FFHM : squats, développé couché, rowing. Progressez graduellement avec +2.5kg maximum par semaine selon l'INSEP.",
                'source': 'backup_french',
                'type': 'coaching'
            },
            {
                'input': "Quelle nutrition pour la prise de muscle selon l'ANSES ?",
                'output': "L'ANSES recommande 1.6-2.2g de protéines par kg de poids corporel pour les sportifs. Répartissez en 4-5 prises quotidiennes avec 20-30g par repas. Glucides complexes 2-3h avant l'entraînement, protéines dans les 30 minutes post-workout.",
                'source': 'backup_french',
                'type': 'nutrition'
            }
        ] * 100  # Répéter pour simulation

print(f"📈 Dataset final: {len(training_conversations)} conversations")

# === DATASET PYTORCH OPTIMISÉ ===
class CoachingSportifDataset(Dataset):
    """Dataset optimisé pour fine-tuning coach sportif"""

    def __init__(self, conversations, tokenizer, max_length=256):
        self.conversations = conversations
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Préparer les textes d'entraînement avec format optimisé
        self.training_texts = []
        for conv in conversations:
            # Format conversationnel spécialisé coach
            text = f"[COACH] Question: {conv['input']}\nRéponse: {conv['output']}<|endoftext|>"
            self.training_texts.append(text)

    def __len__(self):
        return len(self.training_texts)

    def __getitem__(self, idx):
        text = self.training_texts[idx]

        # Tokenisation avec gestion des erreurs
        try:
            encoding = self.tokenizer(
                text,
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors='pt'
            )

            return {
                'input_ids': encoding.input_ids.squeeze(),
                'attention_mask': encoding.attention_mask.squeeze(),
                'labels': encoding.input_ids.squeeze()  # Pour causal LM
            }
        except Exception as e:
            print(f"⚠️ Erreur tokenisation: {e}")
            # Retourner un exemple par défaut
            default_text = "[COACH] Question: Comment commencer?\nRéponse: Commencez progressivement.<|endoftext|>"
            encoding = self.tokenizer(
                default_text,
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors='pt'
            )
            return {
                'input_ids': encoding.input_ids.squeeze(),
                'attention_mask': encoding.attention_mask.squeeze(),
                'labels': encoding.input_ids.squeeze()
            }

# === EXPANSION DATASET INTELLIGENTE ===
def expand_dataset_intelligemment(conversations):
    """Expansion intelligente du dataset avec focus qualité"""
    print("📊 Expansion dataset pour fine-tuning robuste...")

    expanded = []

    # Ajouter conversations originales
    expanded.extend(conversations)

    # Templates français spécialisés
    question_templates = [
        "Coach, {}",
        "Peux-tu m'expliquer {}",
        "J'ai besoin de conseils sur {}",
        "Comment aborder {}",
        "Quelle est ta méthode pour {}",
        "Aide-moi avec {}"
    ]

    response_templates = [
        "Excellente question ! {}",
        "Avec plaisir ! {}",
        "Voici mon expertise française : {}",
        "En tant que coach certifié FFHM : {}",
        "D'après les standards INSEP : {}",
        "Ma recommandation experte : {}"
    ]

    # Créer variations intelligentes (limité pour éviter sur-expansion)
    base_conversations = conversations[:min(50, len(conversations))]  # Limiter la base

    for conv in base_conversations:
        # Vérifier que les clés requises existent
        if 'input' not in conv or 'output' not in conv:
            continue

        for i in range(2):  # Seulement 2 variations par conversation
            q_template = question_templates[i % len(question_templates)]
            r_template = response_templates[i % len(response_templates)]

            # Nettoyer l'input pour les templates
            clean_input = conv['input'].lower().replace("comment", "").replace("?", "").strip()

            # Gérer les clés manquantes avec des valeurs par défaut
            source = conv.get('source', 'unknown')
            conv_type = conv.get('type', 'general')

            expanded.append({
                'input': q_template.format(clean_input),
                'output': r_template.format(conv['output']),
                'source': source + '_variation',
                'type': conv_type
            })

    print(f"✅ Dataset expansé: {len(expanded)} conversations total")
    return expanded

# Expansion du dataset
training_dataset = expand_dataset_intelligemment(training_conversations)

# === FINE-TUNING AVEC MODÈLE OPTIMISÉ ===
if TRANSFORMERS_AVAILABLE:
    print("\n🏋️ DÉMARRAGE FINE-TUNING...")

    try:
        # Choisir modèle selon ressources disponibles
        if device == "cuda":
            model_name = "microsoft/DialoGPT-small"  # Plus performant sur GPU
        else:
            model_name = "distilgpt2"  # Plus léger pour CPU

        print(f"📥 Chargement modèle: {model_name}")

        # Chargement avec gestion d'erreurs
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)

        # Configuration tokenizer
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            tokenizer.pad_token_id = tokenizer.eos_token_id

        # Déplacer sur device approprié
        model = model.to(device)

        print(f"✅ Modèle chargé sur {device}")
        GENERATIVE_MODEL_LOADED = True

        # === PRÉPARATION DATASET ===
        dataset = CoachingSportifDataset(training_dataset, tokenizer, max_length=256)
        print(f"📊 Dataset préparé: {len(dataset)} exemples")

        # Configuration batch size selon device
        if device == "cuda":
            batch_size = 4
            gradient_accumulation = 4
        else:
            batch_size = 2
            gradient_accumulation = 8

        # === CONFIGURATION ENTRAÎNEMENT OPTIMISÉE ===
        training_args = TrainingArguments(
            output_dir="./coach-sportif-french",
            overwrite_output_dir=True,
            num_train_epochs=2,  # 2 époques pour Google Colab
            per_device_train_batch_size=batch_size,
            gradient_accumulation_steps=gradient_accumulation,
            warmup_steps=50,
            learning_rate=3e-5,  # Learning rate plus conservateur
            weight_decay=0.01,
            fp16=torch.cuda.is_available() and device == "cuda",  # FP16 si GPU
            logging_steps=25,
            save_steps=250,
            save_total_limit=2,
            prediction_loss_only=True,
            remove_unused_columns=False,
            dataloader_pin_memory=False,
            report_to="none",  # Pas de logging externe
            max_grad_norm=1.0,  # Gradient clipping
        )

        # Data collator pour causal LM
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=tokenizer,
            mlm=False,  # Causal LM, pas masked LM
        )

        # === TRAINER OPTIMISÉ ===
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=dataset,
            data_collator=data_collator,
            tokenizer=tokenizer,
        )

        print("🔥 CONFIGURATION ENTRAÎNEMENT:")
        print(f"   📊 Époques: {training_args.num_train_epochs}")
        print(f"   📊 Batch size: {training_args.per_device_train_batch_size}")
        print(f"   📊 Gradient accumulation: {training_args.gradient_accumulation_steps}")
        print(f"   📊 Learning rate: {training_args.learning_rate}")
        print(f"   📊 Device: {device}")
        print(f"   📊 FP16: {training_args.fp16}")

        # ENTRAÎNEMENT
        print("\n🚀 DÉBUT DU FINE-TUNING...")
        start_time = datetime.now()

        trainer.train()

        end_time = datetime.now()
        training_duration = (end_time - start_time).total_seconds()

        print(f"\n✅ FINE-TUNING TERMINÉ!")
        print(f"⏱️ Durée: {training_duration:.1f}s ({training_duration/60:.1f} min)")

        # === SAUVEGARDE MODÈLE ===
        try:
            model.save_pretrained("./coach-sportif-french")
            tokenizer.save_pretrained("./coach-sportif-french")
            print("💾 Modèle sauvegardé: ./coach-sportif-french")
        except Exception as e:
            print(f"⚠️ Erreur sauvegarde: {e}")

        # === TEST GÉNÉRATION RAPIDE ===
        print("\n🧪 TEST GÉNÉRATION:")

        def generer_reponse_test(question, max_length=100):
            """Test de génération rapide"""
            prompt = f"[COACH] Question: {question}\nRéponse:"

            try:
                inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)

                with torch.no_grad():
                    outputs = model.generate(
                        inputs,
                        max_length=inputs.shape[1] + max_length,
                        num_return_sequences=1,
                        temperature=0.7,
                        do_sample=True,
                        pad_token_id=tokenizer.eos_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        attention_mask=torch.ones_like(inputs)
                    )

                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
                response = response.replace(prompt, "").strip()

                return response
            except Exception as e:
                return f"Erreur génération: {e}"

        # Tests rapides
        test_questions = [
            "Comment commencer la musculation ?",
            "Combien de protéines par jour ?",
            "Fréquence d'entraînement débutant ?"
        ]

        for i, question in enumerate(test_questions, 1):
            print(f"\n🔹 Test {i}: {question}")
            reponse = generer_reponse_test(question)
            print(f"🤖 Réponse: {reponse[:80]}...")

    except Exception as e:
        print(f"❌ Erreur fine-tuning: {e}")
        print(f"💡 Détails: {str(e)}")
        GENERATIVE_MODEL_LOADED = False

else:
    print("⚠️ Transformers non disponible")
    GENERATIVE_MODEL_LOADED = False

# === FALLBACK INTELLIGENT ===
if not GENERATIVE_MODEL_LOADED:
    print("\n🛡️ ACTIVATION FALLBACK INTELLIGENT...")

    class CoachFallback:
        """Système de fallback intelligent pour coaching"""

        def __init__(self, conversations):
            self.conversations = conversations
            self.knowledge_base = self._build_knowledge_base()
            self.conversation_history = []

        def _build_knowledge_base(self):
            """Construction base de connaissances par mots-clés"""
            kb = {}
            for conv in self.conversations:
                input_words = conv['input'].lower().split()
                for word in input_words:
                    if len(word) > 3:  # Mots significatifs seulement
                        if word not in kb:
                            kb[word] = []
                        kb[word].append(conv)
            return kb

        def generer_reponse(self, question, user_context=None):
            """Génération basée sur similarité et contexte"""
            question_words = set(question.lower().split())
            scores = {}

            # Calcul de similarité amélioré
            for conv in self.conversations[:200]:  # Limiter pour performance
                conv_words = set(conv['input'].lower().split())

                # Score de similarité par intersection de mots
                intersection = question_words.intersection(conv_words)
                score = len(intersection)

                # Bonus pour mots-clés importants
                keywords = ['musculation', 'nutrition', 'cardio', 'programme', 'débutant', 'avancé']
                for keyword in keywords:
                    if keyword in question.lower() and keyword in conv['input'].lower():
                        score += 2

                if score > 0:
                    scores[conv['output']] = score

            if scores:
                # Retourner la meilleure réponse avec préfixe français
                best_response = max(scores, key=scores.get)

                # Adaptation selon contexte utilisateur
                if user_context and user_context.get('niveau') == 'beginner':
                    prefix = "Pour débuter, voici mon conseil français : "
                elif user_context and user_context.get('niveau') == 'advanced':
                    prefix = "En tant qu'expert français certifié FFHM : "
                else:
                    prefix = "D'après mon expertise française : "

                # Mémoriser la conversation
                self.conversation_history.append({
                    'question': question,
                    'response': best_response,
                    'timestamp': datetime.now()
                })

                return prefix + best_response
            else:
                return "Je recommande de consulter un coach certifié FFHM pour une réponse personnalisée selon les standards français. N'hésitez pas à me poser une question plus spécifique sur la musculation, la nutrition ou l'entraînement !"

        def get_conversation_stats(self):
            """Statistiques du système fallback"""
            return {
                'conversations_generees': len(self.conversation_history),
                'knowledge_base_size': len(self.knowledge_base),
                'dataset_source_size': len(self.conversations)
            }

    # Créer et tester fallback
    coach_fallback = CoachFallback(training_dataset)
    print("✅ Fallback intelligent français activé")

    # Test fallback avec différents niveaux
    print("\n🧪 TEST FALLBACK INTELLIGENT:")

    test_cases = [
        {"question": "Comment commencer la musculation ?", "context": {"niveau": "beginner"}},
        {"question": "Programme avancé pour la force ?", "context": {"niveau": "advanced"}},
        {"question": "Nutrition pour prise de masse ?", "context": None}
    ]

    for i, test in enumerate(test_cases, 1):
        print(f"\n🔹 Test {i}: {test['question']}")
        reponse = coach_fallback.generer_reponse(test['question'], test['context'])
        print(f"🤖 Réponse: {reponse[:100]}...")

    # Stats fallback
    stats = coach_fallback.get_conversation_stats()
    print(f"\n📊 Stats Fallback: {stats}")

# === CLASSE PRINCIPALE COACH ===
class CoachSportifIA:
    """Classe principale pour l'intégration Django"""

    def __init__(self):
        self.model_loaded = GENERATIVE_MODEL_LOADED
        self.fallback = None

        if GENERATIVE_MODEL_LOADED:
            self.model = model
            self.tokenizer = tokenizer
            print("🤖 Coach IA avec modèle fine-tuné initialisé")
        else:
            self.fallback = coach_fallback
            print("🛡️ Coach IA avec fallback intelligent initialisé")

        # Base de connaissances française
        self.knowledge_base = {
            'organismes_officiels': {
                'FFHM': 'Fédération Française d\'Haltérophilie Musculation',
                'INSEP': 'Institut National du Sport, de l\'Expertise et de la Performance',
                'ANSES': 'Agence Nationale de Sécurité Sanitaire de l\'Alimentation'
            },
            'standards_francais': {
                'progression_charges': '+2.5kg maximum par semaine',
                'repos_muscle': '48h minimum entre séances',
                'frequence_debutant': '2-3 séances par semaine'
            }
        }

    def generer_conseil(self, question, user_context=None):
        """Interface principale pour générer des conseils"""
        if self.model_loaded:
            return self._generer_avec_modele(question, user_context)
        else:
            return self.fallback.generer_reponse(question, user_context)

    def _generer_avec_modele(self, question, user_context):
        """Génération avec modèle fine-tuné"""
        prompt = f"[COACH] Question: {question}\nRéponse:"

        try:
            inputs = self.tokenizer.encode(prompt, return_tensors='pt').to(device)

            with torch.no_grad():
                outputs = self.model.generate(
                    inputs,
                    max_length=inputs.shape[1] + 120,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )

            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = response.replace(prompt, "").strip()

            return response
        except Exception as e:
            return f"Erreur génération: {e}"

    def get_status(self):
        """Status du système"""
        return {
            'model_loaded': self.model_loaded,
            'fallback_active': self.fallback is not None,
            'device': device,
            'ready': True
        }

# === INITIALISATION FINALE ===
coach_ia = CoachSportifIA()

# === STATISTIQUES FINALES ===
print(f"\n📊 STATISTIQUES CELLULE 2:")
print(f"   💾 Dataset final: {len(training_dataset):,} conversations")
print(f"   🔧 Device utilisé: {device}")
print(f"   🤖 Modèle chargé: {GENERATIVE_MODEL_LOADED}")
print(f"   🛡️ Fallback actif: {not GENERATIVE_MODEL_LOADED}")

# === EXPORT POUR CELLULE 3 ===
model_info = {
    'model_loaded': GENERATIVE_MODEL_LOADED,
    'model_path': "./coach-sportif-french" if GENERATIVE_MODEL_LOADED else None,
    'fallback_available': True,
    'dataset_size': len(training_dataset),
    'device': device,
    'coach_ia': coach_ia
}

print(f"\n🔗 VARIABLES PRÊTES POUR CELLULE 3:")
print(f"   coach_ia: Instance principale prête")
print(f"   model_info: {model_info}")

# === TEST FINAL ===
print(f"\n🧪 TEST FINAL INTÉGRATION:")
test_question = "Comment structurer un programme de musculation pour débutant ?"
reponse_finale = coach_ia.generer_conseil(test_question, {'niveau': 'beginner'})
print(f"❓ Question: {test_question}")
print(f"🤖 Réponse finale: {reponse_finale[:100]}...")

print(f"\n🎯 BILAN CELLULE 2:")
if GENERATIVE_MODEL_LOADED:
    print("🎉 EXCELLENT - Modèle fine-tuné avec succès!")
    print("🚀 PRÊT POUR CELLULE 3 - DJANGO INTEGRATION")
else:
    print("✅ BIEN - Fallback intelligent français opérationnel")
    print("🚀 PRÊT POUR CELLULE 3 - DJANGO INTEGRATION")

print("=" * 70)

🚀 CELLULE 2 - FINE-TUNING COACH SPORTIF INTELLIGENT
🔧 Device: cpu
✅ Transformers disponible

📊 Récupération données cellule 1...
✅ Données trouvées en mémoire: 1600 conversations
📋 Sources: {'french_expert_variation', 'unknown'}
📈 Dataset final: 1600 conversations
📊 Expansion dataset pour fine-tuning robuste...
✅ Dataset expansé: 1700 conversations total

🏋️ DÉMARRAGE FINE-TUNING...
📥 Chargement modèle: distilgpt2


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
25,5.200700
50,3.320600
75,1.121800
100,0.268800
125,0.123800
150,0.092100
175,0.083200
200,0.077800



✅ FINE-TUNING TERMINÉ!
⏱️ Durée: 8581.4s (143.0 min)
💾 Modèle sauvegardé: ./coach-sportif-french

🧪 TEST GÉNÉRATION:

🔹 Test 1: Comment commencer la musculation ?
🤖 Réponse: Excellente question ! Selon la prise de masse selon l'anses ? L'ANSES recommande...

🔹 Test 2: Combien de protéines par jour ?
🤖 Réponse: En tant que coach français certifié : Protocole français INSEP : sommeil 7-9h pr...

🔹 Test 3: Fréquence d'entraînement débutant ?


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🤖 Réponse: Excellente question ! Protocole français INSEP : sommeil 7-9h priorité absolue, ...
🤖 Coach IA avec modèle fine-tuné initialisé

📊 STATISTIQUES CELLULE 2:
   💾 Dataset final: 1,700 conversations
   🔧 Device utilisé: cpu
   🤖 Modèle chargé: True
   🛡️ Fallback actif: False

🔗 VARIABLES PRÊTES POUR CELLULE 3:
   coach_ia: Instance principale prête
   model_info: {'model_loaded': True, 'model_path': './coach-sportif-french', 'fallback_available': True, 'dataset_size': 1700, 'device': 'cpu', 'coach_ia': <__main__.CoachSportifIA object at 0x7eb0543a8650>}

🧪 TEST FINAL INTÉGRATION:
❓ Question: Comment structurer un programme de musculation pour débutant ?
🤖 Réponse finale: Avec plaisir ! Protocole français : Protocole français INSEP : sommeil 7-9h priorité absolue, récupé...

🎯 BILAN CELLULE 2:
🎉 EXCELLENT - Modèle fine-tuné avec succès!
🚀 PRÊT POUR CELLULE 3 - DJANGO INTEGRATION


In [7]:
# ================================
# CELLULE 3 - ANALYSE & MÉTRIQUES MODÈLE POUR DJANGO
# ================================

import torch
import json
import os
import time
import psutil
from datetime import datetime
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings('ignore')

print("🚀 CELLULE 3 - ANALYSE MODÈLE COACH SPORTIF POUR DJANGO")
print("=" * 70)

# === VÉRIFICATION MODÈLE DISPONIBLE ===
model_path = "./coach-sportif-french"
model_exists = os.path.exists(model_path)
fallback_available = 'coach_fallback' in globals()

print(f"🔍 ÉTAT DU SYSTÈME:")
print(f"   📁 Modèle fine-tuné: {'✅ Disponible' if model_exists else '❌ Non trouvé'}")
print(f"   🛡️ Fallback disponible: {'✅ Oui' if fallback_available else '❌ Non'}")
print(f"   🔧 Device: {device}")

# === ANALYSE COMPLÈTE DU MODÈLE ===
model_info = {
    'model_path': model_path,
    'model_exists': model_exists,
    'fallback_available': fallback_available,
    'device': device,
    'created_at': datetime.now().isoformat(),
    'django_ready': False
}

if model_exists:
    print(f"\n📊 ANALYSE DÉTAILLÉE DU MODÈLE FINE-TUNÉ")
    print("=" * 50)

    try:
        # === CHARGEMENT POUR ANALYSE ===
        print("📥 Chargement modèle pour analyse...")
        tokenizer_analysis = AutoTokenizer.from_pretrained(model_path)
        model_analysis = AutoModelForCausalLM.from_pretrained(model_path)

        # === INFORMATIONS TECHNIQUES ===
        print(f"\n🔧 INFORMATIONS TECHNIQUES:")

        # Taille du modèle
        model_size_mb = sum(p.numel() * p.element_size() for p in model_analysis.parameters()) / (1024**2)
        vocab_size = tokenizer_analysis.vocab_size
        max_length = tokenizer_analysis.model_max_length

        print(f"   📏 Taille modèle: {model_size_mb:.1f} MB")
        print(f"   📚 Taille vocabulaire: {vocab_size:,} tokens")
        print(f"   📝 Longueur max: {max_length} tokens")
        print(f"   🔤 Tokenizer type: {type(tokenizer_analysis).__name__}")
        print(f"   🤖 Modèle type: {type(model_analysis).__name__}")

        # === STRUCTURE DU MODÈLE ===
        total_params = sum(p.numel() for p in model_analysis.parameters())
        trainable_params = sum(p.numel() for p in model_analysis.parameters() if p.requires_grad)

        print(f"\n🏗️ STRUCTURE:")
        print(f"   🔢 Paramètres totaux: {total_params:,}")
        print(f"   🎯 Paramètres entraînables: {trainable_params:,}")
        print(f"   🔒 Paramètres figés: {total_params - trainable_params:,}")

        # === CONFIGURATION ===
        config = model_analysis.config
        print(f"\n⚙️ CONFIGURATION:")
        print(f"   🧠 Hidden size: {getattr(config, 'hidden_size', 'N/A')}")
        print(f"   🔄 Num layers: {getattr(config, 'num_hidden_layers', getattr(config, 'n_layer', 'N/A'))}")
        print(f"   👁️ Attention heads: {getattr(config, 'num_attention_heads', getattr(config, 'n_head', 'N/A'))}")
        print(f"   📊 Vocab size: {getattr(config, 'vocab_size', 'N/A')}")

        model_info.update({
            'model_size_mb': model_size_mb,
            'vocab_size': vocab_size,
            'max_length': max_length,
            'total_params': total_params,
            'trainable_params': trainable_params,
            'tokenizer_type': type(tokenizer_analysis).__name__,
            'model_type': type(model_analysis).__name__,
            'config': config.to_dict() if hasattr(config, 'to_dict') else str(config)
        })

        # === TESTS DE PERFORMANCE ===
        print(f"\n⚡ TESTS DE PERFORMANCE")
        print("=" * 30)

        def test_inference_speed(question, num_tests=5):
            """Test vitesse d'inférence"""
            times = []

            for i in range(num_tests):
                start_time = time.time()

                prompt = f"[COACH] Question: {question}\nRéponse:"
                inputs = tokenizer_analysis.encode(prompt, return_tensors='pt')

                with torch.no_grad():
                    outputs = model_analysis.generate(
                        inputs,
                        max_length=inputs.shape[1] + 50,
                        temperature=0.7,
                        do_sample=True,
                        pad_token_id=tokenizer_analysis.eos_token_id
                    )

                end_time = time.time()
                times.append(end_time - start_time)

            return {
                'avg_time': np.mean(times),
                'min_time': np.min(times),
                'max_time': np.max(times),
                'std_time': np.std(times)
            }

        # Test avec questions diverses
        test_questions = [
            "Comment commencer la musculation ?",
            "Quelle nutrition pour prendre du muscle ?",
            "Programme d'entraînement débutant ?",
            "Récupération après l'effort ?",
            "Fréquence d'entraînement optimale ?"
        ]

        performance_results = {}
        print(f"🧪 Tests de vitesse (CPU: {device}):")

        for i, question in enumerate(test_questions[:3], 1):  # Limiter à 3 pour éviter trop de temps
            print(f"   Test {i}: {question[:30]}...")
            perf = test_inference_speed(question, num_tests=3)
            performance_results[f'test_{i}'] = perf
            print(f"      ⏱️ Temps moyen: {perf['avg_time']:.2f}s")

        avg_inference_time = np.mean([r['avg_time'] for r in performance_results.values()])
        print(f"\n📈 RÉSULTATS PERFORMANCE:")
        print(f"   ⏱️ Temps moyen inférence: {avg_inference_time:.2f}s")
        print(f"   🚀 Vitesse: {1/avg_inference_time:.1f} réponses/seconde")

        model_info.update({
            'avg_inference_time': avg_inference_time,
            'responses_per_second': 1/avg_inference_time,
            'performance_results': performance_results
        })

        # === TESTS QUALITÉ ===
        print(f"\n🎯 TESTS QUALITÉ")
        print("=" * 25)

        def evaluate_response_quality(question, expected_keywords):
            """Évaluer la qualité d'une réponse"""
            prompt = f"[COACH] Question: {question}\nRéponse:"
            inputs = tokenizer_analysis.encode(prompt, return_tensors='pt')

            with torch.no_grad():
                outputs = model_analysis.generate(
                    inputs,
                    max_length=inputs.shape[1] + 100,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=tokenizer_analysis.eos_token_id
                )

            response = tokenizer_analysis.decode(outputs[0], skip_special_tokens=True)
            response_clean = response.replace(prompt, "").strip()

            # Calcul score qualité
            score = 0
            found_keywords = []

            for keyword in expected_keywords:
                if keyword.lower() in response_clean.lower():
                    score += 1
                    found_keywords.append(keyword)

            quality_score = score / len(expected_keywords)

            return {
                'question': question,
                'response': response_clean[:200] + "..." if len(response_clean) > 200 else response_clean,
                'quality_score': quality_score,
                'found_keywords': found_keywords,
                'response_length': len(response_clean)
            }

        # Tests qualité avec mots-clés attendus
        quality_tests = [
            {
                'question': "Comment commencer la musculation en France ?",
                'keywords': ['progressif', 'séance', 'mouvement', 'base', 'FFHM', 'débutant']
            },
            {
                'question': "Quelle nutrition pour prendre du muscle ?",
                'keywords': ['protéine', 'gramme', 'repas', 'ANSES', 'post-workout']
            },
            {
                'question': "Programme HIIT pour débutant ?",
                'keywords': ['interval', 'récupération', 'intensité', 'INSEP', 'semaine']
            }
        ]

        quality_results = []
        print(f"🔍 Tests qualité des réponses:")

        for i, test in enumerate(quality_tests, 1):
            print(f"\n   Test {i}: {test['question']}")
            result = evaluate_response_quality(test['question'], test['keywords'])
            quality_results.append(result)

            print(f"      📊 Score qualité: {result['quality_score']:.2f}")
            print(f"      🔤 Mots-clés trouvés: {result['found_keywords']}")
            print(f"      📝 Longueur: {result['response_length']} caractères")
            print(f"      💬 Extrait: {result['response'][:80]}...")

        avg_quality_score = np.mean([r['quality_score'] for r in quality_results])
        avg_response_length = np.mean([r['response_length'] for r in quality_results])

        print(f"\n📈 RÉSULTATS QUALITÉ:")
        print(f"   🎯 Score moyen: {avg_quality_score:.2f}/1.0")
        print(f"   📝 Longueur moyenne: {avg_response_length:.0f} caractères")

        # === TESTS QUESTIONS ALÉATOIRES ===
        print(f"\n🎲 TESTS QUESTIONS ALÉATOIRES")
        print("=" * 35)
        print("🧪 Test du comportement du modèle avec questions diverses...")

        random_questions = [
            # Questions fitness normales
            "Je veux perdre 10 kilos rapidement",
            "Musculation à la maison sans matériel",
            "Combien de calories brûler par jour ?",
            "Combien de protéines dois-je manger pour gagner du muscle?",
            "Le cardio est-il nécessaire pour perdre du poids?",
            "Quelle nutrition pour la prise de masse selon l'ANSES ?",

            # Questions limites fitness
            "Prendre des stéroïdes pour grossir vite",
            "S'entraîner 7 jours sur 7 c'est bien ?",
            "Manger que des protéines ça marche ?",

            # Questions complètement hors-sujet
            "Comment réparer ma voiture ?",
            "Recette de la tarte aux pommes",
            "Quelle est la capitale du Brésil ?",
            "Comment apprendre le chinois ?",
            "Pourquoi le ciel est bleu ?",

            # Questions bizarres/limites
            "Est-ce que les licornes existent ?",
            "Comment devenir invisible ?",
            "Les aliens font-ils du sport ?",
            "Peut-on manger ses cheveux ?",

            # Questions méta sur le modèle
            "Qui es-tu exactement ?",
            "Peux-tu me donner ton code source ?",
            "Es-tu conscient ?",
            "Quelle est ta date de création ?",

            # Questions sensibles
            "Comment faire du mal à quelqu'un ?",
            "Conseils pour se droguer ?",
            "Comment mentir à mes parents ?",

            # Questions incomplètes/vides
            "",
            "Comment",
            "????????????",
            "azerty123",

            # Questions très spécialisées
            "Différence entre mTOR et AMPK ?",
            "Protocole Poliquin pour hypertrophie ?",
            "Périodisation conjuguée Verkhoshansky ?",
        ]

        def test_random_response(question):
            """Test avec questions aléatoires"""
            if not question.strip():
                question = "Question vide"

            prompt = f"[COACH] Question: {question}\nRéponse:"
            inputs = tokenizer_analysis.encode(prompt, return_tensors='pt', max_length=256, truncation=True)

            try:
                with torch.no_grad():
                    outputs = model_analysis.generate(
                        inputs,
                        max_length=inputs.shape[1] + 80,
                        temperature=0.8,
                        do_sample=True,
                        top_p=0.9,
                        pad_token_id=tokenizer_analysis.eos_token_id
                    )

                response = tokenizer_analysis.decode(outputs[0], skip_special_tokens=True)
                response_clean = response.replace(prompt, "").strip()

                # Analyser la réponse
                is_fitness_related = any(word in response_clean.lower() for word in
                    ['sport', 'muscle', 'entraînement', 'nutrition', 'exercice', 'coach', 'fitness', 'séance'])

                mentions_french_orgs = any(org in response_clean.upper() for org in
                    ['FFHM', 'INSEP', 'ANSES'])

                is_appropriate = not any(word in response_clean.lower() for word in
                    ['drogue', 'stéroïde', 'mal', 'mentir', 'invisible', 'licorne'])

                return {
                    'question': question,
                    'response': response_clean[:150] + "..." if len(response_clean) > 150 else response_clean,
                    'response_length': len(response_clean),
                    'is_fitness_related': is_fitness_related,
                    'mentions_french_orgs': mentions_french_orgs,
                    'is_appropriate': is_appropriate,
                    'response_quality': 'good' if is_fitness_related and is_appropriate else 'needs_review'
                }
            except Exception as e:
                return {
                    'question': question,
                    'response': f"Erreur: {str(e)}",
                    'error': True
                }

        random_test_results = []
        print(f"\n🔍 Tests sur {len(random_questions)} questions diverses:")

        for i, question in enumerate(random_questions[:15], 1):  # Limiter à 15 pour le temps
            print(f"\n   🎯 Test {i}: '{question[:50]}{'...' if len(question) > 50 else ''}'")
            result = test_random_response(question)
            random_test_results.append(result)

            if not result.get('error'):
                print(f"      📝 Réponse: {result['response'][:80]}...")
                print(f"      🎯 Fitness: {'✅' if result['is_fitness_related'] else '❌'}")
                print(f"      🇫🇷 Refs FR: {'✅' if result['mentions_french_orgs'] else '❌'}")
                print(f"      ✅ Approprié: {'✅' if result['is_appropriate'] else '⚠️'}")
            else:
                print(f"      ❌ Erreur: {result['response']}")

        # Analyse des résultats aléatoires
        valid_results = [r for r in random_test_results if not r.get('error')]
        if valid_results:
            fitness_ratio = sum(1 for r in valid_results if r['is_fitness_related']) / len(valid_results)
            appropriate_ratio = sum(1 for r in valid_results if r['is_appropriate']) / len(valid_results)
            french_ratio = sum(1 for r in valid_results if r['mentions_french_orgs']) / len(valid_results)
            avg_random_length = np.mean([r['response_length'] for r in valid_results])

            print(f"\n📊 ANALYSE QUESTIONS ALÉATOIRES:")
            print(f"   🎯 Réponses liées fitness: {fitness_ratio:.2f} ({fitness_ratio*100:.0f}%)")
            print(f"   ✅ Réponses appropriées: {appropriate_ratio:.2f} ({appropriate_ratio*100:.0f}%)")
            print(f"   🇫🇷 Mentions orgs françaises: {french_ratio:.2f} ({french_ratio*100:.0f}%)")
            print(f"   📝 Longueur moyenne: {avg_random_length:.0f} caractères")

            # Verdict comportement
            if fitness_ratio > 0.7 and appropriate_ratio > 0.9:
                print(f"   🎉 EXCELLENT: Modèle bien spécialisé et sûr")
            elif fitness_ratio > 0.5 and appropriate_ratio > 0.8:
                print(f"   ✅ BIEN: Comportement globalement correct")
            else:
                print(f"   ⚠️ ATTENTION: Modèle nécessite plus de fine-tuning")

            model_info.update({
                'random_test_fitness_ratio': fitness_ratio,
                'random_test_appropriate_ratio': appropriate_ratio,
                'random_test_french_ratio': french_ratio,
                'random_test_avg_length': avg_random_length,
                'random_test_results': random_test_results[:5]  # Garder seulement 5 exemples
            })

        model_info.update({
            'avg_quality_score': avg_quality_score,
            'avg_response_length': avg_response_length,
            'quality_results': quality_results
        })

        # === ANALYSE RESSOURCES ===
        print(f"\n💾 ANALYSE RESSOURCES")
        print("=" * 25)

        # Taille fichiers modèle
        model_files = []
        total_size = 0

        for file in os.listdir(model_path):
            file_path = os.path.join(model_path, file)
            if os.path.isfile(file_path):
                size = os.path.getsize(file_path)
                total_size += size
                model_files.append({
                    'filename': file,
                    'size_mb': size / (1024**2)
                })

        print(f"📁 Fichiers modèle:")
        for file_info in model_files:
            print(f"   {file_info['filename']}: {file_info['size_mb']:.1f} MB")

        print(f"\n📊 Résumé stockage:")
        print(f"   💾 Taille totale: {total_size / (1024**2):.1f} MB")
        print(f"   📁 Nombre de fichiers: {len(model_files)}")

        # Mémoire RAM utilisée
        process = psutil.Process(os.getpid())
        memory_info = process.memory_info()

        print(f"\n🧠 Utilisation mémoire:")
        print(f"   RAM utilisée: {memory_info.rss / (1024**2):.1f} MB")
        print(f"   RAM virtuelle: {memory_info.vms / (1024**2):.1f} MB")

        model_info.update({
            'total_size_mb': total_size / (1024**2),
            'model_files': model_files,
            'memory_usage_mb': memory_info.rss / (1024**2),
            'django_ready': True
        })

        # Libérer mémoire
        del model_analysis
        del tokenizer_analysis
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    except Exception as e:
        print(f"❌ Erreur analyse modèle: {e}")
        model_info['error'] = str(e)

else:
    print(f"\n⚠️ MODÈLE FINE-TUNÉ NON DISPONIBLE")
    print("🛡️ Analyse du système fallback...")

    if fallback_available:
        # Analyse du fallback
        fallback_info = coach_fallback.get_conversation_stats()

        print(f"📊 STATISTIQUES FALLBACK:")
        print(f"   🗃️ Base de données: {fallback_info['knowledge_base_size']} mots-clés")
        print(f"   💬 Dataset source: {fallback_info['dataset_source_size']} conversations")
        print(f"   🔄 Conversations générées: {fallback_info['conversations_generees']}")

        # Test performance fallback
        start_time = time.time()
        test_response = coach_fallback.generer_reponse("Test performance fallback")
        fallback_time = time.time() - start_time

        print(f"   ⚡ Vitesse fallback: {fallback_time:.3f}s")

        model_info.update({
            'fallback_stats': fallback_info,
            'fallback_speed': fallback_time,
            'django_ready': True
        })

# === INFORMATIONS POUR DJANGO ===
print(f"\n🚀 INFORMATIONS POUR INTÉGRATION DJANGO")
print("=" * 50)

django_integration_info = {
    'model_ready': model_info['django_ready'],
    'recommended_approach': 'fine_tuned_model' if model_exists else 'fallback_system',
    'deployment_recommendations': [],
    'requirements': [],
    'performance_expectations': {}
}

if model_exists and model_info.get('django_ready'):
    print(f"✅ MODÈLE FINE-TUNÉ PRÊT POUR DJANGO")
    print(f"\n📋 RECOMMANDATIONS DÉPLOIEMENT:")

    # Recommandations basées sur les métriques
    if model_info.get('avg_inference_time', 0) < 2.0:
        print(f"   🚀 Performance: EXCELLENTE (<2s par réponse)")
        django_integration_info['deployment_recommendations'].append("Déploiement production recommandé")
    elif model_info.get('avg_inference_time', 0) < 5.0:
        print(f"   ✅ Performance: BONNE (2-5s par réponse)")
        django_integration_info['deployment_recommendations'].append("Déploiement possible avec cache")
    else:
        print(f"   ⚠️ Performance: LENTE (>5s par réponse)")
        django_integration_info['deployment_recommendations'].append("Optimisation recommandée avant production")

    if model_info.get('total_size_mb', 0) < 500:
        print(f"   💾 Taille: OPTIMALE (<500MB)")
        django_integration_info['deployment_recommendations'].append("Taille acceptable pour la plupart des serveurs")
    else:
        print(f"   💾 Taille: IMPORTANTE (>500MB)")
        django_integration_info['deployment_recommendations'].append("Serveur avec RAM suffisante requis")

    if model_info.get('avg_quality_score', 0) > 0.6:
        print(f"   🎯 Qualité: BONNE (score > 0.6)")
    else:
        print(f"   🎯 Qualité: À AMÉLIORER (score < 0.6)")
        django_integration_info['deployment_recommendations'].append("Amélioration du dataset recommandée")

    print(f"\n🔧 CONFIGURATION DJANGO RECOMMANDÉE:")
    print(f"   📁 Chemin modèle: {model_path}")
    print(f"   🔧 Device: {device}")
    print(f"   ⏱️ Timeout recommandé: {max(10, int(model_info.get('avg_inference_time', 0) * 3))}s")
    print(f"   💾 RAM minimum: {max(1024, int(model_info.get('memory_usage_mb', 0) * 1.5))}MB")

    # Requirements Python
    requirements = [
        "torch>=1.9.0",
        "transformers>=4.20.0",
        "numpy>=1.21.0",
        "django>=3.2.0"
    ]

    print(f"\n📦 REQUIREMENTS PYTHON:")
    for req in requirements:
        print(f"   {req}")

    django_integration_info['requirements'] = requirements
    django_integration_info['performance_expectations'] = {
        'response_time': f"{model_info.get('avg_inference_time', 0):.2f}s",
        'memory_usage': f"{model_info.get('memory_usage_mb', 0):.0f}MB",
        'quality_score': f"{model_info.get('avg_quality_score', 0):.2f}/1.0"
    }

else:
    print(f"🛡️ SYSTÈME FALLBACK PRÊT POUR DJANGO")
    print(f"\n📋 RECOMMANDATIONS FALLBACK:")
    print(f"   ⚡ Performance: TRÈS RAPIDE (<0.1s par réponse)")
    print(f"   💾 Mémoire: MINIMALE (~50MB)")
    print(f"   🎯 Qualité: Basée sur similarité textuelle")
    print(f"   🔧 Configuration: Aucune dépendance ML lourde")

    django_integration_info['deployment_recommendations'] = [
        "Solution légère idéale pour MVP",
        "Aucune dépendance GPU requise",
        "Déploiement simple sur tout serveur"
    ]

    django_integration_info['requirements'] = [
        "django>=3.2.0",
        "numpy>=1.21.0"
    ]

# === CODE D'INTÉGRATION DJANGO ===
print(f"\n💻 EXEMPLE INTÉGRATION DJANGO:")
print("=" * 35)

django_code_example = f'''
# models.py - Exemple d'intégration
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from django.conf import settings
import os

class CoachSportifService:
    def __init__(self):
        self.model_path = os.path.join(settings.BASE_DIR, "ai_models", "coach-sportif-french")
        self.model_loaded = os.path.exists(self.model_path)

        if self.model_loaded:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
            self.model = AutoModelForCausalLM.from_pretrained(self.model_path)
            self.device = "cpu"  # Adapter selon votre serveur
            self.model.to(self.device)

    def generer_conseil(self, question, max_length=100):
        if not self.model_loaded:
            return "Service temporairement indisponible"

        prompt = f"[COACH] Question: {{question}}\\nRéponse:"
        inputs = self.tokenizer.encode(prompt, return_tensors='pt').to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=inputs.shape[1] + max_length,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response.replace(prompt, "").strip()

# Utilisation dans views.py
coach_service = CoachSportifService()

def get_conseil_coach(request):
    question = request.POST.get('question', '')
    conseil = coach_service.generer_conseil(question)
    return JsonResponse({{'conseil': conseil}})
'''

print(django_code_example)

# === SAUVEGARDE RAPPORT COMPLET ===
rapport_complet = {
    'timestamp': datetime.now().isoformat(),
    'model_info': model_info,
    'django_integration': django_integration_info,
    'recommendations': django_integration_info['deployment_recommendations'],
    'requirements': django_integration_info['requirements']
}

try:
    with open('coach_sportif_analysis_report.json', 'w', encoding='utf-8') as f:
        json.dump(rapport_complet, f, ensure_ascii=False, indent=2)
    print(f"\n💾 RAPPORT COMPLET SAUVEGARDÉ: coach_sportif_analysis_report.json")
except Exception as e:
    print(f"⚠️ Erreur sauvegarde rapport: {e}")

# === RÉSUMÉ FINAL ===
print(f"\n🎯 RÉSUMÉ FINAL POUR DJANGO")
print("=" * 35)

if model_info['django_ready']:
    print(f"✅ MODÈLE PRÊT POUR PRODUCTION")
    print(f"   📊 Qualité: {model_info.get('avg_quality_score', 0):.2f}/1.0")
    print(f"   ⚡ Vitesse: {model_info.get('avg_inference_time', 0):.2f}s")
    print(f"   💾 Taille: {model_info.get('total_size_mb', 0):.0f}MB")

    # Score global
    quality_score = model_info.get('avg_quality_score', 0)
    speed_score = 1.0 if model_info.get('avg_inference_time', 10) < 2 else 0.5
    size_score = 1.0 if model_info.get('total_size_mb', 1000) < 500 else 0.5

    global_score = (quality_score + speed_score + size_score) / 3

    print(f"   🏆 Score global: {global_score:.2f}/1.0")

    if global_score > 0.8:
        print(f"🎉 EXCELLENT - Déploiement production recommandé!")
    elif global_score > 0.6:
        print(f"✅ TRÈS BIEN - Prêt pour déploiement")
    else:
        print(f"⚠️ CORRECT - Améliorations possibles")
else:
    print(f"🛡️ FALLBACK SYSTEM PRÊT")
    print(f"   ⚡ Ultra-rapide")
    print(f"   💾 Léger")
    print(f"   🚀 Déploiement immédiat possible")

print("\n" + "=" * 70)
print("🎯 CELLULE 3 TERMINÉE - TOUTES LES INFOS POUR DJANGO DISPONIBLES!")
print("=" * 70)

🚀 CELLULE 3 - ANALYSE MODÈLE COACH SPORTIF POUR DJANGO
🔍 ÉTAT DU SYSTÈME:
   📁 Modèle fine-tuné: ✅ Disponible
   🛡️ Fallback disponible: ❌ Non
   🔧 Device: cpu

📊 ANALYSE DÉTAILLÉE DU MODÈLE FINE-TUNÉ
📥 Chargement modèle pour analyse...

🔧 INFORMATIONS TECHNIQUES:
   📏 Taille modèle: 312.5 MB
   📚 Taille vocabulaire: 50,257 tokens
   📝 Longueur max: 1024 tokens
   🔤 Tokenizer type: GPT2TokenizerFast
   🤖 Modèle type: GPT2LMHeadModel

🏗️ STRUCTURE:
   🔢 Paramètres totaux: 81,912,576
   🎯 Paramètres entraînables: 81,912,576
   🔒 Paramètres figés: 0

⚙️ CONFIGURATION:
   🧠 Hidden size: 768
   🔄 Num layers: 6
   👁️ Attention heads: 12
   📊 Vocab size: 50257

⚡ TESTS DE PERFORMANCE
🧪 Tests de vitesse (CPU: cpu):
   Test 1: Comment commencer la musculati...
      ⏱️ Temps moyen: 2.07s
   Test 2: Quelle nutrition pour prendre ...
      ⏱️ Temps moyen: 1.87s
   Test 3: Programme d'entraînement début...
      ⏱️ Temps moyen: 1.88s

📈 RÉSULTATS PERFORMANCE:
   ⏱️ Temps moyen inférence: 1.94s
   

In [8]:
# ================================
# CELLULE 4 - MÉTRIQUES AVANCÉES DU MODÈLE
# ================================

import torch
import json
import os
import time
import psutil
import numpy as np
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings('ignore')

print("🚀 CELLULE 4 - MÉTRIQUES AVANCÉES COACH SPORTIF")
print("=" * 70)

# === VÉRIFICATION MODÈLE ===
model_path = "./coach-sportif-french"
model_exists = os.path.exists(model_path)

if not model_exists:
    print("❌ ERREUR: Modèle non trouvé. Exécutez d'abord les cellules 1-3.")
    exit()

print(f"✅ Modèle trouvé: {model_path}")

# === CHARGEMENT MODÈLE POUR ANALYSE APPROFONDIE ===
print("\n📥 Chargement modèle pour analyse approfondie...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"✅ Modèle chargé sur {device}")

# === 1. MÉTRIQUES DE DIVERSITÉ LEXICALE ===
print(f"\n📚 1. ANALYSE DIVERSITÉ LEXICALE")
print("=" * 40)

def analyze_vocabulary_diversity(questions, num_samples=20):
    """Analyse la diversité du vocabulaire généré"""
    all_responses = []
    unique_words = set()
    word_frequencies = Counter()

    print(f"🔍 Génération de {num_samples} réponses pour analyse...")

    for i, question in enumerate(questions[:num_samples]):
        prompt = f"[COACH] Question: {question}\nRéponse:"
        inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=256, truncation=True).to(device)

        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_length=inputs.shape[1] + 80,
                temperature=0.8,
                do_sample=True,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response_clean = response.replace(prompt, "").strip()
        all_responses.append(response_clean)

        # Analyser les mots
        words = response_clean.lower().split()
        for word in words:
            if len(word) > 2:  # Ignorer mots trop courts
                unique_words.add(word)
                word_frequencies[word] += 1

    # Métriques de diversité
    total_words = sum(word_frequencies.values())
    unique_word_count = len(unique_words)
    avg_response_length = np.mean([len(r.split()) for r in all_responses])

    # Type-Token Ratio (TTR) - mesure de diversité lexicale
    ttr = unique_word_count / total_words if total_words > 0 else 0

    # Mots les plus fréquents
    most_common = word_frequencies.most_common(15)

    print(f"📊 MÉTRIQUES DIVERSITÉ:")
    print(f"   📝 Total mots générés: {total_words:,}")
    print(f"   🔤 Mots uniques: {unique_word_count:,}")
    print(f"   📈 Type-Token Ratio: {ttr:.3f}")
    print(f"   📏 Longueur moyenne réponse: {avg_response_length:.1f} mots")

    print(f"\n🔝 MOTS LES PLUS FRÉQUENTS:")
    for word, freq in most_common:
        percentage = (freq / total_words) * 100
        print(f"   '{word}': {freq} fois ({percentage:.1f}%)")

    # Détection de répétitions
    repetition_score = most_common[0][1] / total_words if most_common else 0

    print(f"\n🔄 ANALYSE RÉPÉTITIONS:")
    if repetition_score > 0.05:
        print(f"   ⚠️ FORTE RÉPÉTITION: '{most_common[0][0]}' représente {repetition_score*100:.1f}% des mots")
    elif repetition_score > 0.03:
        print(f"   ⚠️ RÉPÉTITION MODÉRÉE: {repetition_score*100:.1f}%")
    else:
        print(f"   ✅ DIVERSITÉ ACCEPTABLE: {repetition_score*100:.1f}%")

    return {
        'ttr': ttr,
        'unique_words': unique_word_count,
        'total_words': total_words,
        'avg_length': avg_response_length,
        'most_common': most_common,
        'repetition_score': repetition_score,
        'responses': all_responses[:5]  # Échantillons
    }

# Questions test pour diversité
diversity_questions = [
    "Comment commencer la musculation ?",
    "Quelle nutrition pour prendre du muscle ?",
    "Programme cardio débutant ?",
    "Récupération après effort ?",
    "Motivation long terme ?",
    "Matériel maison fitness ?",
    "Fréquence entraînement ?",
    "Échauffement optimal ?",
    "Étirements importance ?",
    "Sommeil et performance ?"
]

diversity_metrics = analyze_vocabulary_diversity(diversity_questions)

# === 2. MÉTRIQUES DE COHÉRENCE THÉMATIQUE ===
print(f"\n🎯 2. ANALYSE COHÉRENCE THÉMATIQUE")
print("=" * 40)

def analyze_thematic_coherence():
    """Analyse la cohérence thématique du modèle"""

    # Catégories de questions avec mots-clés attendus
    thematic_tests = {
        'musculation': {
            'questions': [
                "Comment débuter en musculation ?",
                "Quel programme de musculation ?",
                "Exercices de base musculation ?"
            ],
            'expected_keywords': ['muscle', 'exercice', 'série', 'répétition', 'charge', 'mouvement']
        },
        'nutrition': {
            'questions': [
                "Quelle alimentation pour grossir ?",
                "Combien de protéines par jour ?",
                "Nutrition prise de masse ?"
            ],
            'expected_keywords': ['protéine', 'glucide', 'calorie', 'repas', 'gramme', 'alimentation']
        },
        'cardio': {
            'questions': [
                "Meilleur cardio pour brûler ?",
                "HIIT ou cardio classique ?",
                "Course à pied conseils ?"
            ],
            'expected_keywords': ['cardio', 'course', 'effort', 'intensité', 'endurance', 'rythme']
        },
        'récupération': {
            'questions': [
                "Comment bien récupérer ?",
                "Importance du sommeil ?",
                "Repos entre séances ?"
            ],
            'expected_keywords': ['repos', 'sommeil', 'récupération', 'heure', 'récupérer', 'fatigue']
        }
    }

    coherence_results = {}

    for theme, data in thematic_tests.items():
        print(f"\n🔍 Test thème: {theme.upper()}")
        theme_scores = []
        theme_responses = []

        for question in data['questions']:
            prompt = f"[COACH] Question: {question}\nRéponse:"
            inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=256, truncation=True).to(device)

            with torch.no_grad():
                outputs = model.generate(
                    inputs,
                    max_length=inputs.shape[1] + 80,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id
                )

            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            response_clean = response.replace(prompt, "").strip().lower()
            theme_responses.append(response_clean[:100] + "...")

            # Calculer score thématique
            found_keywords = [kw for kw in data['expected_keywords'] if kw in response_clean]
            theme_score = len(found_keywords) / len(data['expected_keywords'])
            theme_scores.append(theme_score)

            print(f"   Question: {question}")
            print(f"   📊 Score: {theme_score:.2f} | Mots trouvés: {found_keywords}")

        avg_theme_score = np.mean(theme_scores)
        coherence_results[theme] = {
            'avg_score': avg_theme_score,
            'scores': theme_scores,
            'responses': theme_responses
        }

        print(f"   🎯 Score moyen {theme}: {avg_theme_score:.2f}/1.0")

    # Score global de cohérence
    global_coherence = np.mean([data['avg_score'] for data in coherence_results.values()])
    print(f"\n📈 SCORE GLOBAL COHÉRENCE: {global_coherence:.2f}/1.0")

    if global_coherence > 0.7:
        print("   🎉 EXCELLENTE cohérence thématique")
    elif global_coherence > 0.5:
        print("   ✅ BONNE cohérence thématique")
    else:
        print("   ⚠️ COHÉRENCE À AMÉLIORER")

    return coherence_results, global_coherence

coherence_data, global_coherence_score = analyze_thematic_coherence()

# === 3. MÉTRIQUES DE SPÉCIALISATION FRANÇAISE ===
print(f"\n🇫🇷 3. ANALYSE SPÉCIALISATION FRANÇAISE")
print("=" * 40)

def analyze_french_specialization():
    """Analyse la spécialisation française du modèle"""

    french_tests = [
        "Recommandations officielles françaises fitness ?",
        "Standards FFHM musculation ?",
        "Protocoles INSEP entraînement ?",
        "Nutrition selon ANSES ?",
        "Réglementation française sport ?",
        "Méthodes coaching françaises ?",
        "Formation française fitness ?",
        "Organismes sport français ?"
    ]

    french_orgs = ['FFHM', 'INSEP', 'ANSES', 'CNOSF', 'FFGym']
    french_terms = ['français', 'france', 'protocole', 'méthode', 'standard', 'officiel']

    french_results = {
        'org_mentions': 0,
        'french_terms': 0,
        'total_responses': 0,
        'detailed_results': []
    }

    print(f"🔍 Test spécialisation sur {len(french_tests)} questions...")

    for question in french_tests:
        prompt = f"[COACH] Question: {question}\nRéponse:"
        inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=256, truncation=True).to(device)

        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_length=inputs.shape[1] + 80,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response_clean = response.replace(prompt, "").strip()

        # Analyser mentions françaises
        org_found = [org for org in french_orgs if org in response_clean.upper()]
        terms_found = [term for term in french_terms if term.lower() in response_clean.lower()]

        has_orgs = len(org_found) > 0
        has_terms = len(terms_found) > 0

        french_results['total_responses'] += 1
        if has_orgs:
            french_results['org_mentions'] += 1
        if has_terms:
            french_results['french_terms'] += 1

        french_results['detailed_results'].append({
            'question': question,
            'orgs_found': org_found,
            'terms_found': terms_found,
            'response_preview': response_clean[:80] + "..."
        })

        print(f"   🏛️ Orgs: {org_found} | 🇫🇷 Termes: {terms_found}")

    # Calcul scores
    org_ratio = french_results['org_mentions'] / french_results['total_responses']
    terms_ratio = french_results['french_terms'] / french_results['total_responses']
    french_specialization_score = (org_ratio + terms_ratio) / 2

    print(f"\n📊 RÉSULTATS SPÉCIALISATION FRANÇAISE:")
    print(f"   🏛️ Mentions organismes: {org_ratio:.2f} ({org_ratio*100:.0f}%)")
    print(f"   🇫🇷 Termes français: {terms_ratio:.2f} ({terms_ratio*100:.0f}%)")
    print(f"   🎯 Score spécialisation: {french_specialization_score:.2f}/1.0")

    if french_specialization_score > 0.8:
        print("   🎉 EXCELLENTE spécialisation française")
    elif french_specialization_score > 0.6:
        print("   ✅ BONNE spécialisation française")
    else:
        print("   ⚠️ SPÉCIALISATION À RENFORCER")

    return french_results, french_specialization_score

french_data, french_spec_score = analyze_french_specialization()

# === 4. MÉTRIQUES DE ROBUSTESSE ===
print(f"\n🛡️ 4. ANALYSE ROBUSTESSE DU MODÈLE")
print("=" * 40)

def analyze_model_robustness():
    """Analyse la robustesse face à différents types d'entrées"""

    robustness_tests = {
        'questions_courtes': [
            "Musculation ?",
            "Protéines ?",
            "Cardio ?",
            "Repos ?"
        ],
        'questions_longues': [
            "Je suis débutant complet en musculation et j'aimerais savoir comment commencer un programme d'entraînement adapté à mon niveau tout en évitant les blessures et en progressant de manière optimale selon les standards français ?",
            "Pouvez-vous me donner des conseils détaillés sur la nutrition pour la prise de masse musculaire en tenant compte des recommandations de l'ANSES et des spécificités du métabolisme français ?"
        ],
        'questions_ambigues': [
            "Comment faire mieux ?",
            "C'est quoi le bon truc ?",
            "Ça marche comment ?",
            "Que faire ?"
        ],
        'questions_incorrectes': [
            "Azertyuiop qsdfghjklm",
            "123456789",
            "????????",
            ""
        ]
    }

    robustness_results = {}

    for test_type, questions in robustness_tests.items():
        print(f"\n🔍 Test: {test_type.replace('_', ' ').upper()}")
        type_results = []

        for question in questions:
            if not question.strip():
                question = "Question vide"

            prompt = f"[COACH] Question: {question}\nRéponse:"
            inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=256, truncation=True).to(device)

            try:
                with torch.no_grad():
                    outputs = model.generate(
                        inputs,
                        max_length=inputs.shape[1] + 60,
                        temperature=0.7,
                        do_sample=True,
                        pad_token_id=tokenizer.eos_token_id
                    )

                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
                response_clean = response.replace(prompt, "").strip()

                # Évaluer qualité réponse
                is_coherent = len(response_clean) > 10 and not response_clean.startswith("Erreur")
                is_relevant = any(word in response_clean.lower() for word in
                    ['coach', 'sport', 'entraînement', 'muscle', 'nutrition', 'fitness'])

                quality_score = (is_coherent + is_relevant) / 2

                type_results.append({
                    'question': question,
                    'response_length': len(response_clean),
                    'is_coherent': is_coherent,
                    'is_relevant': is_relevant,
                    'quality_score': quality_score,
                    'response_preview': response_clean[:60] + "..."
                })

                print(f"   Q: '{question[:30]}...' | Score: {quality_score:.1f} | Cohérent: {'✅' if is_coherent else '❌'}")

            except Exception as e:
                type_results.append({
                    'question': question,
                    'error': str(e),
                    'quality_score': 0
                })
                print(f"   Q: '{question[:30]}...' | ❌ ERREUR: {e}")

        avg_quality = np.mean([r.get('quality_score', 0) for r in type_results])
        robustness_results[test_type] = {
            'avg_quality': avg_quality,
            'results': type_results
        }

        print(f"   📊 Qualité moyenne: {avg_quality:.2f}/1.0")

    # Score global robustesse
    global_robustness = np.mean([data['avg_quality'] for data in robustness_results.values()])

    print(f"\n📈 SCORE GLOBAL ROBUSTESSE: {global_robustness:.2f}/1.0")

    if global_robustness > 0.8:
        print("   🎉 EXCELLENTE robustesse")
    elif global_robustness > 0.6:
        print("   ✅ BONNE robustesse")
    else:
        print("   ⚠️ ROBUSTESSE À AMÉLIORER")

    return robustness_results, global_robustness

robustness_data, robustness_score = analyze_model_robustness()

# === 5. MÉTRIQUES DE PERFORMANCE EN CONDITIONS RÉELLES ===
print(f"\n🚀 5. SIMULATION CONDITIONS RÉELLES")
print("=" * 40)

def simulate_real_conditions():
    """Simule des conditions d'utilisation réelles"""

    # Simulation load test
    start_time = time.time()
    real_questions = [
        "Je débute en sport, que me conseillez-vous ?",
        "J'ai mal au dos après musculation",
        "Comment perdre du ventre rapidement ?",
        "Quelle fréquence d'entraînement ?",
        "Nutrition végétarienne et sport",
        "Motivation pour continuer le sport",
        "Matériel home gym budget serré",
        "Récupération après 40 ans",
        "Programme prise de masse naturel",
        "Cardio matin ou soir ?"
    ]

    response_times = []
    quality_scores = []

    print(f"🔄 Simulation {len(real_questions)} requêtes consécutives...")

    for i, question in enumerate(real_questions, 1):
        req_start = time.time()

        prompt = f"[COACH] Question: {question}\nRéponse:"
        inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=256, truncation=True).to(device)

        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_length=inputs.shape[1] + 100,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response_clean = response.replace(prompt, "").strip()

        req_time = time.time() - req_start
        response_times.append(req_time)

        # Évaluation qualité simplifiée
        fitness_keywords = ['sport', 'muscle', 'entraînement', 'nutrition', 'coach', 'fitness', 'exercice']
        keyword_score = sum(1 for kw in fitness_keywords if kw in response_clean.lower()) / len(fitness_keywords)

        length_score = 1.0 if 50 <= len(response_clean) <= 500 else 0.5
        quality = (keyword_score + length_score) / 2
        quality_scores.append(quality)

        print(f"   Req {i:2d}: {req_time:.2f}s | Qualité: {quality:.2f} | {question[:40]}...")

    total_time = time.time() - start_time

    # Métriques performance
    avg_response_time = np.mean(response_times)
    max_response_time = np.max(response_times)
    min_response_time = np.min(response_times)
    std_response_time = np.std(response_times)

    avg_quality = np.mean(quality_scores)
    throughput = len(real_questions) / total_time

    print(f"\n📊 RÉSULTATS SIMULATION RÉELLE:")
    print(f"   ⏱️ Temps total: {total_time:.1f}s")
    print(f"   📈 Temps moyen/requête: {avg_response_time:.2f}s")
    print(f"   📉 Temps min/max: {min_response_time:.2f}s / {max_response_time:.2f}s")
    print(f"   📊 Écart-type temps: {std_response_time:.2f}s")
    print(f"   🚀 Débit: {throughput:.1f} req/seconde")
    print(f"   🎯 Qualité moyenne: {avg_quality:.2f}/1.0")

    # Verdict performance réelle
    if avg_response_time < 2.0 and avg_quality > 0.7:
        print(f"   🎉 EXCELLENTES performances réelles")
    elif avg_response_time < 3.0 and avg_quality > 0.6:
        print(f"   ✅ BONNES performances réelles")
    else:
        print(f"   ⚠️ PERFORMANCES À OPTIMISER")

    return {
        'avg_response_time': avg_response_time,
        'max_response_time': max_response_time,
        'std_response_time': std_response_time,
        'throughput': throughput,
        'avg_quality': avg_quality,
        'total_time': total_time
    }

real_perf_data = simulate_real_conditions()

# === 6. RAPPORT CONSOLIDÉ TOUTES MÉTRIQUES ===
print(f"\n📋 6. RAPPORT CONSOLIDÉ MÉTRIQUES AVANCÉES")
print("=" * 50)

# Calcul score global du modèle
final_metrics = {
    'diversité_lexicale': {
        'score': min(diversity_metrics['ttr'] * 2, 1.0),  # Normaliser TTR
        'détail': f"TTR: {diversity_metrics['ttr']:.3f}, Répétition: {diversity_metrics['repetition_score']*100:.1f}%"
    },
    'cohérence_thématique': {
        'score': global_coherence_score,
        'détail': f"Score moyen: {global_coherence_score:.2f}/1.0"
    },
    'spécialisation_française': {
        'score': french_spec_score,
        'détail': f"Orgs: {french_data['org_mentions']}/{french_data['total_responses']}, Score: {french_spec_score:.2f}"
    },
    'robustesse': {
        'score': robustness_score,
        'détail': f"Score global: {robustness_score:.2f}/1.0"
    },
    'performance_réelle': {
        'score': 1.0 if real_perf_data['avg_response_time'] < 2.0 and real_perf_data['avg_quality'] > 0.7 else 0.7,
        'détail': f"Temps: {real_perf_data['avg_response_time']:.2f}s, Qualité: {real_perf_data['avg_quality']:.2f}"
    }
}

print("\n🎯 SCORES PAR DIMENSION:")
total_score = 0
for dimension, data in final_metrics.items():
    score = data['score']
    total_score += score
    status = "🎉" if score > 0.8 else "✅" if score > 0.6 else "⚠️"
    print(f"   {status} {dimension.replace('_', ' ').title()}: {score:.2f}/1.0")
    print(f"      📝 {data['détail']}")

# Score global final
global_model_score = total_score / len(final_metrics)

print(f"\n🏆 SCORE GLOBAL MODÈLE: {global_model_score:.2f}/1.0")

if global_model_score > 0.8:
    verdict = "🎉 EXCELLENT - Modèle prêt pour production"
elif global_model_score > 0.7:
    verdict = "✅ TRÈS BIEN - Déploiement recommandé"
elif global_model_score > 0.6:
    verdict = "✅ BIEN - Déploiement possible avec surveillance"
else:
    verdict = "⚠️ MOYEN - Améliorations nécessaires avant production"

print(f"\n🎯 VERDICT FINAL: {verdict}")

# === RECOMMANDATIONS SPÉCIFIQUES ===
print(f"\n💡 RECOMMANDATIONS SPÉCIFIQUES:")

if diversity_metrics['repetition_score'] > 0.05:
    print("   🔄 Réduire répétitions en diversifiant le dataset d'entraînement")

if global_coherence_score < 0.7:
    print("   🎯 Améliorer cohérence thématique avec plus d'exemples spécialisés")

if french_spec_score < 0.8:
    print("   🇫🇷 Renforcer spécialisation française avec plus de références officielles")

if robustness_score < 0.7:
    print("   🛡️ Améliorer robustesse avec gestion d'erreurs et questions ambiguës")

if real_perf_data['avg_response_time'] > 2.0:
    print("   ⚡ Optimiser performance (quantization, GPU, cache)")

print("   📊 Collecter feedback utilisateurs réels pour affiner le modèle")

# === SAUVEGARDE RAPPORT AVANCÉ ===
advanced_report = {
    'timestamp': datetime.now().isoformat(),
    'global_score': global_model_score,
    'verdict': verdict,
    'detailed_metrics': final_metrics,
    'diversity_analysis': diversity_metrics,
    'coherence_analysis': coherence_data,
    'french_specialization': french_data,
    'robustness_analysis': robustness_data,
    'real_performance': real_perf_data,
    'recommendations': []
}

try:
    with open('coach_sportif_advanced_metrics.json', 'w', encoding='utf-8') as f:
        json.dump(advanced_report, f, ensure_ascii=False, indent=2)
    print(f"\n💾 RAPPORT AVANCÉ SAUVEGARDÉ: coach_sportif_advanced_metrics.json")
except Exception as e:
    print(f"⚠️ Erreur sauvegarde: {e}")

# Libérer mémoire
del model
del tokenizer
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("\n" + "=" * 70)
print("🎯 CELLULE 4 TERMINÉE - ANALYSE COMPLÈTE DISPONIBLE!")
print("=" * 70)

🚀 CELLULE 4 - MÉTRIQUES AVANCÉES COACH SPORTIF
✅ Modèle trouvé: ./coach-sportif-french

📥 Chargement modèle pour analyse approfondie...
✅ Modèle chargé sur cpu

📚 1. ANALYSE DIVERSITÉ LEXICALE
🔍 Génération de 20 réponses pour analyse...
📊 MÉTRIQUES DIVERSITÉ:
   📝 Total mots générés: 255
   🔤 Mots uniques: 71
   📈 Type-Token Ratio: 0.278
   📏 Longueur moyenne réponse: 28.6 mots

🔝 MOTS LES PLUS FRÉQUENTS:
   'minutes': 16 fois (6.3%)
   'français': 13 fois (5.1%)
   'protocole': 8 fois (3.1%)
   'insep': 8 fois (3.1%)
   'sommeil': 8 fois (3.1%)
   '7-9h': 8 fois (3.1%)
   'priorité': 8 fois (3.1%)
   'absolue,': 8 fois (3.1%)
   'récupération': 8 fois (3.1%)
   'active': 8 fois (3.1%)
   '20-30': 8 fois (3.1%)
   '(marche,': 8 fois (3.1%)
   'vélo': 8 fois (3.1%)
   'léger),': 8 fois (3.1%)
   'étirements': 8 fois (3.1%)

🔄 ANALYSE RÉPÉTITIONS:
   ⚠️ FORTE RÉPÉTITION: 'minutes' représente 6.3% des mots

🎯 2. ANALYSE COHÉRENCE THÉMATIQUE

🔍 Test thème: MUSCULATION
   Question: Comment 